# Redis Assignment

The purpose of this task is to create the cookie syncronization service between two companies with the help of Redis.

In [4]:
import redis
import time
from datetime import timedelta

class LimitExceededException(Exception):
    pass

max_rps = 15  # requests per second

# Basic database settings. Don't change this line!
r = redis.StrictRedis(host='localhost', port=6381, db=0) 

In [5]:
def make_key(first, second):
    return str(first)+":"+str(second)

def save_sync(r, uid, partner_id, partner_uid):
    """
        Set the values for the pairs <partner_id, uid> and <partner_id, partner_uid>
        Don't forget to set the ttls which you defined in the function set_ttls
    """

    r.hmset(str(partner_id), {'user:'+uid:partner_uid})
    r.hmset(str(partner_id), {'partner:'+partner_uid:uid})
    
    set_ttls(r, {partner_id : 3600})

def get_partner_uid(r, uid, partner_id):
    """Get the partner id by the pair (uid, partner id)
    
    Args:
        r (redis.StrictRedis): redis instance
        uid (str): our uid
        partner_id (str): id of the partner
        
    Examples:
        >>> get_partner_uid(r, 'e5a370cc-6bdc-43ae-baaa-8fd4531847f7', 12)
    """
    
    if not r.get('get_partner_uid'):
        set_ttls(r, {'get_partner_uid' : 1})
     
    r.incrby('get_partner_uid', 1)    

    if(int(r.get('get_partner_uid'))>max_rps):
        raise LimitExceededException
 

    return r.hget(str(partner_id), str('user:'+uid))



def get_uid(r, partner_id, partner_uid):
    """Get the uid by the pair (partner id, partner uid)
    
    Args:
        r (redis.StrictRedis): redis instance
        partner_id (int): id of partner
        partner_uid (str): uid of partner
        
    Examples:
        >>> get_uid(r, 12, '25b6e9a6-fca8-427c-94df-2577e62b2bf0')
    """
    if not r.get('get_uid'):
        set_ttls(r, {'get_uid' : 1})
    
    r.incrby('get_uid', 1)    
    
    if(int(r.get('get_uid'))>max_rps):
        raise LimitExceededException
        
    return r.hget(str(partner_id), str('partner:'+partner_uid))


def set_ttls(r, ttls):
    """Set the ttl by partner id
       
    Args:
        r (redis.StrictRedis): redis instance
        ttls (dict): dictionary of pairs <partner_id, ttl>
        
    Examples:
        >>> set_ttls(r, {12: 5, 3: 0.25})
    """
    pipe = r.pipeline()
    for key, value in ttls.items():
        pipe.pexpire(str(key), int(value*1000))
    pipe.execute()

There are some basic checks for your function

In [8]:
# Usage examples


save_sync(r, 'e5a370cc-6bdc-43ae-baaa-8fd4531847f7', 12, '25b6e9a6-fca8-427c-94df-2577e62b2bf0')
set_ttls(r, {12: 0.25, 3: 0.25})

# Some basic checks

assert 'e5a370cc-6bdc-43ae-baaa-8fd4531847f7' == get_uid(r, 12, '25b6e9a6-fca8-427c-94df-2577e62b2bf0').decode('utf-8')

assert '25b6e9a6-fca8-427c-94df-2577e62b2bf0' == get_partner_uid(r, 'e5a370cc-6bdc-43ae-baaa-8fd4531847f7', 12).decode('utf-8')


Your code will be tested by calling the combination of functions:
* `set_ttls`
* `save_sync`
* `get_partner_uid`
* `get_uid`

You should check that the `get` functions return `None` when the ttl has been expired. You can implement this check with the help of the function `time.sleep(your_defined_ttl)`

Congratulations! You have completed the assignment! Now you can submit it to the system and get your results!